In [1]:
import azureml.core
from azureml.core import Workspace, Experiment
from azureml.pipeline.core import Pipeline
from azureml.pipeline.steps import DatabricksStep
from azureml.core.compute.databricks import DatabricksCompute, ComputeTarget
import dotenv
%load_ext dotenv
%dotenv
import os

In [2]:
ws = Workspace.from_config()

WARNING - Warning: Falling back to use azure cli login credentials.
If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


In [3]:
compute_config = DatabricksCompute.attach_configuration(
        resource_group = os.getenv("RESOURCE_GROUP"),
        workspace_name = os.getenv("DATABRICKS_WORKSPACE_NAME"),
        access_token= os.getenv("DATABRICKS_ACCESS_TOKEN"))

In [5]:
databricks_compute=ComputeTarget.attach(ws, "dbcompute", compute_config)
databricks_compute.wait_for_completion(True)

Creating...
SucceededProvisioning operation finished, operation "Succeeded"


In [7]:
step = DatabricksStep(
    name="scoring_step",
    existing_cluster_id=os.getenv("DATABRICKS_CLUSTER_ID"),
    inputs=None,
    outputs=None,
    notebook_path="/Users/anta@microsoft.com/notebooks/3_Scoring_Pipeline",
    notebook_params={
            "results_data": "hpresults_output",
            "model": "RandomForest",
            "start_date": "2015-11-15",
            "to_date": "2016-04-30"
    },
    run_name="test_run",
    timeout_seconds=600,
    compute_target=databricks_compute,
    min_workers=2,
    max_workers=4,
    allow_reuse=False)

In [8]:
steps = [step]
pipeline = Pipeline(workspace=ws, steps=steps)
pipeline_run = Experiment(ws, 'DB_Pipeline').submit(pipeline)
pipeline_run.wait_for_completion()

Created step scoring_step [dda19daa][349c5c3b-d904-41c8-a693-3c087790d6b7], (This step will run and generate new outputs)
Submitted pipeline run: 2a18e0c7-6402-4ff3-87de-d84adeec454e
RunId: 2a18e0c7-6402-4ff3-87de-d84adeec454e
Link to Portal: https://mlworkspace.azure.ai/portal/subscriptions/a61eb99b-265d-430b-91c8-911f86ae9be1/resourceGroups/anta_batchspark/providers/Microsoft.MachineLearningServices/workspaces/antabsws/experiments/DB_Pipeline/runs/2a18e0c7-6402-4ff3-87de-d84adeec454e
Status: Running
...............................................................................................
Status: Finished
{'runId': '2a18e0c7-6402-4ff3-87de-d84adeec454e', 'status': 'Completed', 'startTimeUtc': '2019-05-23T11:39:49.537016Z', 'endTimeUtc': '2019-05-23T11:48:21.838213Z', 'properties': {'azureml.runsource': 'azureml.PipelineRun', 'runSource': None, 'runType': 'HTTP', 'azureml.parameters': '{}'}, 'logFiles': {'logs/azureml/stderrlogs.txt': 'https://antabswsstoraged7bdd7dac.blob.core.w

'Finished'